In [7]:
from pydantic import BaseModel, root_validator


class UserForm(BaseModel):
    username: str | None
    password: str | None


class UserFormValidation(BaseModel):
    valid_username: bool
    valid_password: bool
    valid: bool | None

    @root_validator
    def validate(cls, values) -> dict:
        valid = (
            True
            if values["valid_username"] and values["valid_password"]
            else False
        )
        values["valid"] = valid
        return values


In [10]:
v = UserFormValidation(valid_password=True, valid_username=False)

In [11]:
v.dict()

{'valid_username': False, 'valid_password': True, 'valid': False}